In [ ]:
import tensorflow as tf
import numpy as np 
from sklearn.datasets import fetch_california_housing
from IPython.display import clear_output, Image, display, HTML
from sklearn import preprocessing

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

###### Implement Data Preprocess here ######
housing = fetch_california_housing()
print("Shape of dataset:", housing.data.shape)
print("Shape of label:", housing.target.shape)
m, n = housing.data.shape

train_num = int(m*0.9)
print train_num

X = housing.data
X = np.c_[np.ones((m, 1)),X]
y = housing.target

###### Implement Data Preprocess here ######
trainX = X[:train_num]
trainY = y[:train_num]
trainY = trainY.reshape(-1,1)

testX = X[train_num+1:]
testY = y[train_num+1:]
testY = testY.reshape(-1,1)

# W = (X'X)^(-1)X'Y 
trainX_T = np.transpose(trainX)

W = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(trainX_T, trainX)), trainX_T), trainY)

###### Start TF session ######
with tf.Session() as sess:
    pred = tf.matmul(testX,W)
    print pred.eval()
    err = tf.reduce_mean(tf.abs(pred - testY)/testY)
    print err.eval()
    #show_graph(tf.get_default_graph().as_graph_def())
###### Start TF session ######